# GMN Example

This notebook demonstrates how to register a groundwatermonitoringnetwork (GMN) in the BRO using bro-exchange.

In [52]:
import bro_exchange as brx
from getpass import getpass

Define the token used for authentification (Bronhouderportaal API) below. Note: a token generated in the demo environment should be used. Besides, the project id should be given when using api v2.

In [53]:
user = input('user:')
password = getpass('pass:')
project_id = input('project_id:')


## 1. Register a new GMN

### 1.1. Generate registration request

Get GMN data:

In [54]:
monitoringTube1 = {'broId':'GMW000000026289',
                   'tubeNumber':2}

monitoringTube2 = {'broId':'GMW000000026288',
                   'tubeNumber':3}

measuringPoints = [{'measuringPointCode':'GMW07F000001',
                'monitoringTube':monitoringTube1},
                    {'measuringPointCode':'GMW07F000002',
                'monitoringTube':monitoringTube2}]

srcdocdata =  {'objectIdAccountableParty':'Meetnet_07',
                'name':'Provinciaal meetnet grondwater Groningen; Delfzijl',        
                'deliveryContext':'kaderrichtlijnWater',        
                'monitoringPurpose':'strategischBeheerKwaliteitRegionaal',        
                'groundwaterAspect':'kwantiteit',    
                'startDateMonitoring':['2021-01-01','date'],     
                'measuringPoints':measuringPoints,              
                } 

Generate registartion request (GMN_StartRegistration):

In [55]:
reg = brx.gmn_registration_request(srcdoc='GMN_StartRegistration', requestReference = 'GMN_StartRegistration_test', deliveryAccountableParty = '27376655', qualityRegime = 'IMBRO', srcdocdata=srcdocdata)
reg.generate()
reg.write_request(output_dir = 'output\gmn', filename = 'registration_request_gmn_startregistration_test.xml')


### 1.2. Validate registration request

Validate the generated request (by means of bronhouderportaal api). In this case, the demo environment of the bronhouderportaal is used.

In [56]:
reg.validate(user=user, password=password, api='v2', project_id = 1255, demo=True)

Check if there are some errors:

In [57]:
print(reg.validation_status)
print(reg.validation_info)

VALIDE
{'status': 'VALIDE'}


The xml document seems to be valid

### 1.3. Deliver registration request

Deliver the generared request. For delivering the request, information 

In [61]:
reg.deliver(user=user, password=password, api='v2', project_id = 1255, demo=True)
print(reg.delivery_info.json())
print(reg.delivery_id)

{'identifier': '0000182614', 'status': 'AANGELEVERD', 'lastChanged': '2022-11-15T23:23:57.530097', 'brondocuments': [{'id': 418602, 'status': 'GELDIG', 'lastChanged': '2022-11-15T23:23:57.554417', 'filename': '7ea9f52ddea6.xml', 'requestReference': 'GMN_StartRegistration_test'}]}
0000182614


The request is delivered, and the id is stored. After delivering, the request can't be deliverd again:

In [49]:
reg.deliver(user=user, password=password, api='v2', project_id = 1255, demo=True)

Exception: Request has already been delivered

After the request has been deliverd, the delivery id can be stored to track the status.

In [62]:
delivery_id = reg.delivery_id

### 1.4. Track status of pending delivery

To track the status of a pending delivery, the 'check_status' function of bro-exchange can be used. It only requires the delivery id, api version, environment and the authentication data. If api v2 is used, the project id should also be supplied:

In [64]:
status = brx.check_delivery_status(delivery_id, user=user, password=password, api='v2', project_id = project_id, demo=True)
print(status.json())

{'identifier': '0000182614', 'status': 'DOORGELEVERD', 'lastChanged': '2022-11-15T23:24:03.009142', 'brondocuments': [{'id': 418602, 'status': 'OPGENOMEN_LVBRO', 'lastChanged': '2022-11-15T23:24:02.74789', 'broId': 'GMN000000001076', 'transactionId': 'GMN-000000001403', 'filename': '7ea9f52ddea6.xml', 'requestReference': 'GMN_StartRegistration_test'}]}
